<a href="https://colab.research.google.com/github/paritoshkc/ML_Group_income_predictor/blob/master/ML_Group__income_prediction_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [15]:
#installing LightGBM
pip install lightgbm
pip install --upgrade category_encoders

Requirement already up-to-date: category_encoders in /usr/local/lib/python3.6/dist-packages (2.1.0)


In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb
import seaborn as seabornInstance 
import category_encoders as ce
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error 
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score
from sklearn import preprocessing

In [0]:

def prepareData(dataset,dataWithoutLabels):
  #preparing training data+test data
  
  #Gender
  dataset['Gender']=dataset['Gender'].replace(['unknown','0',np.nan],'UNKOWN')  
  dataset['Gender']=dataset['Gender'].replace(['f'],'female')  
  dataWithoutLabels['Gender']=dataWithoutLabels['Gender'].replace(['unknown','0',np.nan],'UNKOWN')  
  dataWithoutLabels['Gender']=dataWithoutLabels['Gender'].replace(['f'],'female')  

  #Housing
  dataset['Housing Situation']=dataset['Housing Situation'].replace(['nA',0,'0'],'UNKOWN')
  dataWithoutLabels['Housing Situation']=dataWithoutLabels['Housing Situation'].replace(['nA',0,'0'],'UNKOWN')

  #work exo
  dataset['Work Experience in Current Job [years]']=pd.to_numeric(dataset['Work Experience in Current Job [years]'].replace(['#NUM!'],0))
  dataWithoutLabels['Work Experience in Current Job [years]']=pd.to_numeric(dataWithoutLabels['Work Experience in Current Job [years]'].replace(['#NUM!'],0))

  #satisfaction with employer
  dataset['Satisfation with employer']=dataset['Satisfation with employer'].replace([np.nan],'UNKNOWN')
  dataWithoutLabels['Satisfation with employer']=dataWithoutLabels['Satisfation with employer'].replace([np.nan],'UNKOWN')

  #extra income
  dataset['Yearly Income in addition to Salary (e.g. Rental Income)']=pd.to_numeric(dataset['Yearly Income in addition to Salary (e.g. Rental Income)'].map(lambda x: x.rstrip(' EUR')))
  dataWithoutLabels['Yearly Income in addition to Salary (e.g. Rental Income)']=pd.to_numeric(dataWithoutLabels['Yearly Income in addition to Salary (e.g. Rental Income)'].map(lambda x: x.rstrip(' EUR')))

  #Haircolor
  dataset['Hair Color']=dataset['Hair Color'].replace(['unknown','0',np.nan],'UNKOWN')
  dataWithoutLabels['Hair Color']=dataWithoutLabels['Hair Color'].replace(['unknown','0',np.nan],'UNKOWN')
 
  return dataset,dataWithoutLabels


In [32]:

  # plot the Values on a graph
def plotActualVsPredictedIncome(y_test, y_pred):
    df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
    df1 = df.head(25)
    df1.plot(kind='bar',figsize=(10,8))
    plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
    plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
    plt.show()    

def trainModel(X_train, y_train, X_test, y_test):
       
    #Implementing lightgbm    
    trainingRegressor = lgb.LGBMRegressor( num_leaves=50, learning_rate=0.01, n_estimators=10000, objective='tweedie', boosting='gbdt' )
    trainingRegressor.fit(X_train, y_train, eval_set= [(X_test, y_test)] , eval_metric='mae', early_stopping_rounds=5 )
    
    print('Starting predicting...')

    # predict
    y_pred = trainingRegressor.predict(X_test, num_iteration=trainingRegressor.best_iteration_)
       
    
    #evaluating the error
    df = pd.DataFrame({'Test': y_test, 'Prediction': y_pred})

    print("mean_absolute_error: %.2f" % mean_absolute_error(y_test, y_pred))
    
    return trainingRegressor
 

def runTrainedModelOnActualData(regressor, featuresToConsider,dataWithoutLabels, encoder):
    print(featuresToConsider)
    print('dataset',dataWithoutLabels[featuresToConsider].dtypes)
    
    X = dataWithoutLabels[featuresToConsider]
    X = encoder.transform(X)
    print('head',X)
    X = sc.transform(X)
    predictedData = regressor.predict(X, num_iteration=regressor.best_iteration_)
    df = pd.DataFrame({'Predicted': predictedData})

    #write the data to csv
    dataset1 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ML/tcd-ml-1920-group-income-submission.csv')

    print("predicted",predictedData.shape)
    print("dataWithoutLabels",dataWithoutLabels.shape)
    print("dataset",dataset1.shape)

    dataset1['Instance'] = dataWithoutLabels['Instance']
    dataset1['Total Yearly Income [EUR]'] = predictedData
    dataset1.to_csv('/content/drive/My Drive/Colab Notebooks/ML/Submission_file_new.csv', index=False)

#Reading data from CSV
dataset = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ML/Train_Data.csv')
dataWithoutLabels = pd.read_csv(('/content/drive/My Drive/Colab Notebooks/ML/Test_Data.csv'))

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
#preprocessing data on both the datasets

dataset,dataWithoutLabels = prepareData(dataset,dataWithoutLabels)


In [0]:
featuresToConsider = ['Year of Record', 'Housing Situation', 'Crime Level in the City of Employement', 'Work Experience in Current Job [years]', 'Satisfation with employer', 'Gender', 'Age', 'Country', 'Size of City', 'Profession', 'University Degree', 'Body Height [cm]', 'Hair Color', 'Wears Glasses', 'Yearly Income in addition to Salary (e.g. Rental Income)']
y = dataset['Total Yearly Income [EUR]'].values

In [35]:

#Features Dataset
X = dataset[featuresToConsider]

# Target encoding on categorical data
target_encoder = ce.TargetEncoder(cols=['Housing Situation', 'Satisfation with employer', 'Hair Color', 'Gender', 'Country', 'Profession', 'University Degree'])
target_encoder.fit(X, y)
X = target_encoder.transform(X)


df = pd.DataFrame(X)
sc = preprocessing.MinMaxScaler()
X = sc.fit_transform(X)


#divide the training data into test and train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


#get the trained model
regressor = trainModel(X_train, y_train, X_test, y_test)

# Run the model with actual test data
runTrainedModelOnActualData(regressor, featuresToConsider,dataWithoutLabels, target_encoder)

[1]	valid_0's tweedie: 1042.98	valid_0's l1: 71857.1
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's tweedie: 1039.54	valid_0's l1: 71121
[3]	valid_0's tweedie: 1036.16	valid_0's l1: 70395.8
[4]	valid_0's tweedie: 1032.82	valid_0's l1: 69681.8
[5]	valid_0's tweedie: 1029.53	valid_0's l1: 68978.9
[6]	valid_0's tweedie: 1026.28	valid_0's l1: 68285.9
[7]	valid_0's tweedie: 1023.09	valid_0's l1: 67602.7
[8]	valid_0's tweedie: 1019.93	valid_0's l1: 66929.6
[9]	valid_0's tweedie: 1016.83	valid_0's l1: 66265.7
[10]	valid_0's tweedie: 1013.77	valid_0's l1: 65612.8
[11]	valid_0's tweedie: 1010.75	valid_0's l1: 64969.4
[12]	valid_0's tweedie: 1007.78	valid_0's l1: 64335.4
[13]	valid_0's tweedie: 1004.85	valid_0's l1: 63710.1
[14]	valid_0's tweedie: 1001.96	valid_0's l1: 63093.7
[15]	valid_0's tweedie: 999.114	valid_0's l1: 62486.7
[16]	valid_0's tweedie: 996.31	valid_0's l1: 61887.9
[17]	valid_0's tweedie: 993.548	valid_0's l1: 61297.7
[18]	valid_0's tweedie: 990.826	v